# Gemini ve LangChain ile LLM API'larını Çağırma Giriş 🦜🔗

Bu notebook'ta LangChain aracılığıyla LLM API'larını nasıl kullanacağınızı öğreneceksiniz. Örnek olarak Google'ın Gemini API'sını kullanacağız. Bu notebook'un sonunda, LangChain kullanarak API çağrıları yapmayı ve bunu neden yaptığımızı bileceksiniz.

## ⚙️ Kurulum

👉 Kurulum aşamasında oluşturduğumuz `.env` dosyasındaki ortam değişkenlerini yüklemek için aşağıdaki hücreyi çalıştırın:

In [1]:
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

True

👉 Hücrenin çıktısı "`True`" mu? Harika! Artık Gemini API ile kimlik doğrulaması yapmak için kullanılacak bir `GOOGLE_API_KEY` ortam değişkeni kurmuş olduk.

Eğer değilse, yardım isteyin.

## Basit Bir API Çağrısı Yapma

Bu notebook'ta şunların nasıl yapılacağını göstereceğiz:
1. Google'ın kendi kütüphanesini kullanarak API çağrısı yapma.
2. Aynı işlemi LangChain kullanarak yapma.

## Google Generative AI Kütüphanesini Kullanma

In [2]:
from google import genai

In [3]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="What is the capital of France?",
)

`response` nesnesine bir göz atalım.

In [4]:
response.candidates[0].content.parts[0].text

'The capital of France is **Paris**.'

Gerçek cevabı nasıl alabileceğinizi görüyor musunuz?

Neyse ki, cevabı hemen almak için sadece `.text` özelliğini kullanabiliriz. Deneyin.

In [5]:
response.text

'The capital of France is **Paris**.'

Gemini cevaplarını Markdown formatında döndürür. Bunu kullanalım!

In [6]:
from IPython.display import Markdown
Markdown(response.text)

The capital of France is **Paris**.

Oluşturma parametrelerini de değiştirebilirsiniz. `google.genai` kullanarak bunu şu şekilde yaparsınız:

In [8]:
from google import genai
from google.genai import types # We need to import types for the config

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Write a social media post about how much you're learning about transformers.",
    config=types.GenerateContentConfig(
        max_output_tokens=200,
        temperature=1.0
    )
)

In [9]:
Markdown(response.text)

Here are a few options for a social media post about learning about transformers, choose the one that best fits your style and platform!

**Option 1: Enthusiastic & Slightly Technical**

🤯 My brain is officially in "Transformer overload" (in the best way possible!) lately. Diving deep into the world of self-attention, positional encodings, and multi-head attention has been absolutely fascinating. It's like unlocking a whole new level of understanding for how these powerful models process language and beyond. So much to learn, so little time! 🧠✨ #Transformers #DeepLearning #AI #MachineLearning #NLP #Tech

**Option 2: More Casual & Relatable**

Who knew that a single "Transformer" could open up so many new ideas? 🤯 I've been spending a lot of time lately learning about these incredible models, and my mind is blown by what they can do. It's a bit complex, but the more I dig,

Harika. Ancak başka bir API denemek istediğinizi düşünün, örneğin OpenAI'nin veya Anthropic'in?

Onların dokümantasyonlarını incelemek ve tüm kodunuzu onların API'sini kullanacak şekilde yeniden yazmak zorunda kalırsınız. Tabii ki benzer olacaktır, ancak aynı olmayacaktır.

Neyse ki LangChain var!

## LangChain Kullanma 🦜🔗

Neden LangChain kullanırsınız?

1. **Model-Bağımsız Kod**

   LangChain, farklı LLM sağlayıcıları (Google, OpenAI, Anthropic, vb.) arasında minimal kod değişikliği ile geçiş yapmanızı sağlayan soyutlamalar sunar. Google API'sine doğrudan kod yazarsanız, sağlayıcı değiştirmek önemli ölçüde yeniden düzenleme gerektirir.

2. **Birleşik Arayüz**

   LangChain, altta yatan API'den bağımsız olarak farklı LLM sağlayıcıları arasında etkileşimleri standartlaştırır ve tutarlı yöntemler ile yanıt formatları sunar.

3. **Bileşenlerle Çalışabilirlik**

   LangChain'in zincir ve pipeline mimarisi, tüm alt yapıyı kendiniz halletmeden prompt, bellek ve erişim sistemlerini birleştiren karmaşık iş akışları oluşturmayı kolaylaştırır.

4. **Yerleşik Araçlar**

   LangChain, çıktı ayrıştırma, prompt şablonları ve kendiniz uygulamanız gereken diğer yardımcı araçları içerir.

[LangChain'in chat entegrasyonları listesi](https://docs.langchain.com/oss/python/integrations/chat)'ne gidin ve entegrasyon listesine bakın. Favori LLM sağlayıcınızı bulabiliyor musunuz?

Kodumuzda `chat_models.ChatGoogleGenerativeAI` kullanmak istemiyoruz çünkü bu özellikle Gemini için yapılmış. LLM'yi değiştirmek istersek, modeli başlatma şeklimizi değiştirmek zorunda kalırız. Neyse ki LangChain bir modeli başlatmak için daha genel bir yol sunar.

Gemini'yi tekrar kullanalım, ancak şimdi LangChain'in genel Chat Models'ini kullanarak.

👉 [LangChain'in "Models" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/models) sayfasına gidin ve Gemini kullanarak bir chat modelinin nasıl başlatılacağını bulun.

İpuçları:
1. Hemen "Basic Usage" bölümüne gidin.
2. Kullanmak istediğiniz modeli seçerek doğru dokümantasyonu hemen görebilirsiniz.

In [10]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "gemini-2.5-flash-lite",
    model_provider="google_genai",
)

Modelin en temel kullanımı sadece `.invoke()` metodunu kullanmaktır:

In [11]:
response = model.invoke("What is the capital of France?")
response

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c1eb1-9c43-7321-a423-2f5bc1984fb4-0', usage_metadata={'input_tokens': 7, 'output_tokens': 8, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

Yanıta bir göz atalım. Nesnenin tüm öznitelik ve metodlarını içeren `__dict__`'ini güzel şekilde yazdırmak için `pprint()` kullanıyoruz.

In [12]:
from pprint import pprint
pprint(response.__dict__)

{'additional_kwargs': {},
 'content': 'The capital of France is **Paris**.',
 'id': 'lc_run--019c1eb1-9c43-7321-a423-2f5bc1984fb4-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'STOP',
                       'model_name': 'gemini-2.5-flash-lite',
                       'model_provider': 'google_genai',
                       'safety_ratings': []},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {'cache_read': 0},
                    'input_tokens': 7,
                    'output_tokens': 8,
                    'total_tokens': 15}}


Cevabı çıkarın ve görüntüleyin. Markdown formatında olduğunu unutmayın, bu yüzden güzel görünmesini sağlayabilirsiniz.

In [13]:
from IPython.display import Markdown

Markdown(response.content)

The capital of France is **Paris**.

Modelin temperature değerini `.temperature` özniteliğine erişerek kontrol edebilirsiniz. Deneyin:

In [14]:
getattr(model, "temperature", None)

0.7

Modeli kullanmadan önce, özniteliklere yeni değerler atayarak oluşturma parametrelerini de ayarlayabiliriz.

Daha önce Google'ın kütüphanesini kullanarak sosyal medya gönderisi yazmak için yaptığımızın eşdeğerini kodlamaya çalışın.

> _Not_: Normal olarak modelin `max_output_tokens` değerini ayarlayabilmemiz gerekir (modeli başlatırken veya daha sonra özniteliği değiştirerek). _langchain_google_genai_'nin mevcut sürümü (4.1.1) bir [hataya](https://github.com/langchain-ai/langchain-google/issues/1454) sahip ve bu çalışmıyor. Geçici çözüm? `max_output_tokens`'ı `.invoke()` metodunun bir parametresi olarak ayarlayın.

In [15]:
# Set the maximum number of output tokens to 200
max_output_tokens = 200

# Set the temperature to 1.0
model.temperature = 1.0

# Generate a response with the new settings
response = model.invoke(
    "Write a social media post about how much you're learning about transformers.",
    max_output_tokens=max_output_tokens,
)

# Display the response
from IPython.display import Markdown
Markdown(response.content)

Here are a few options for a social media post about learning about transformers, ranging in tone and focus. Choose the one that best fits your style!

**Option 1: Enthusiastic & Slightly Overwhelmed**

> Officially diving deep into the world of Transformers! 🤯 My brain is buzzing with concepts like attention mechanisms, self-attention, and the sheer power of these models. It's like unlocking a new level of understanding in AI. So much to learn, so little time, but absolutely loving every minute! #AI #MachineLearning #Transformers #DeepLearning #NLP #LearningJourney

**Option 2: Focused on the "Aha!" Moments**

> Had a major "aha!" moment today while learning about Transformers! The way they process sequences by considering relationships between all elements is truly revolutionary. Feeling like I'm starting to grasp the "why" behind their incredible performance. So excited to keep building on this! ✨ #ArtificialIntelligence #TransformerModels #

Bunun avantajı? Bu LangChain Chat Model birçok başka API'yi destekleyebilir.

Başka bir modele geçmek için değiştirmeniz gereken tek şeyler:
1. Diğer model için bir API anahtarı alın ve kodunuzda tanımlayın.
2. Modeli başlatırken model ve sağlayıcıyı değiştirin.

### Çoklu Mesajlar

`.invoke()` fonksiyonunu sadece tek bir mesajla kullanmak biraz kısıtlayıcı.

Şu gibi birden fazla mesaj sağlayabilirsiniz:
- `SystemMessage` veya sistem mesajları: modelin nasıl davranacağını söylemek için
- `HumanMessage` veya Kullanıcı mesajları: kullanıcıdan gelen girdi
- `AIMessage` veya Asistan mesajları: modelden gelen yanıt

Bir sosyal medya yazarı yapalım.

Modele nasıl davranacağını açıklayan bir sistem mesajı göndereceğiz. Sonra kullanıcı mesajında, kendimizi sadece yazacağı konuyu vermekle sınırlayabiliriz.

Bunu nasıl yapacağınızı öğrenmek için [LangChain'in "Messages" dokümantasyonu](https://docs.langchain.com/oss/python/langchain/messages)'na bakın.

Sistem mesajı için ilhama mı ihtiyacınız var? İşte başlamanız için temel bir talimat:

```python
"""Sen Üretken AI öğrencisi için gönderiler yazan yaratıcı bir sosyal medya yazarısın.
Gönderilerinde her zaman kelime oyunu ve harekete geçirici çağrı bulunur.
Gönderilerin maksimum 200 karakter uzunluğundadır.
Her zaman emoji kullanırsın.
"""
```

In [16]:
# Import the necessary classes
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import Markdown

# Create a list of messages
messages = [
    SystemMessage(
        content=(
            "Sen Üretken AI öğrencisi için gönderiler yazan yaratıcı bir sosyal medya yazarısın.\n"
            "Gönderilerinde her zaman kelime oyunu ve harekete geçirici çağrı bulunur.\n"
            "Gönderilerin maksimum 200 karakter uzunluğundadır.\n"
            "Her zaman emoji kullanırsın."
        )
    ),
    HumanMessage(content="Bugün transformerları öğreniyorum. Kısa, eğlenceli bir post yaz."),
]

# Generate a response using the list of messages
response = model.invoke(messages, max_output_tokens=200)

# Display the response
Markdown(response.content)

Transformerlar hakkında bilgi ediniyorum, beynim adeta bir "dönüşüm" yaşıyor! 🤖 Öğrenmeye devam edin, siz de "transformer" olun! 😉 #yapayzeka #derinnogrenme #transformer

🏁 Tebrikler! Artık LangChain kullanarak çoklu mesajlarla temel prompt yazma konusunda uzmanlaştınız.